In [1]:
import dotenv
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
dotenv.load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-pro")

c:\Users\moham\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("insea-actuariat.pdf")
docs = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings


vectorstore = FAISS.from_documents(splits, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

template = """You are an assistant for question-answering tasks that helps moroccan students in Baccalauréat \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [8]:
chat_history = ""



for i in range(10):
    query = input()
    if query == "exit":
        break
    response = retrieval_chain.invoke({"input": query})
    print("Gemini Pro:", response["answer"])

Gemini Pro: L'actuariat-finance est un domaine scientifique et pluridisciplinaire qui combine des connaissances avancées en statistique, économétrie, économie financière, mathématiques appliquées, économie, droit, fiscalité, informatique, gestion et management. Les spécialistes de ce domaine sont appelés actuaires ou ingénieurs du risque et sont chargés de proposer des modèles stochastiques basés sur la théorie des probabilités pour gérer l'évolution incertaine de l'environnement assurantiel et financier. Ils interviennent dans des domaines tels que la comptabilité, la fiscalité et le commerce.
Gemini Pro: I'm sorry, I cannot find the answer to the previous question in the provided text.
Gemini Pro: Je n'ai pas assez d'informations dans le contexte fourni pour répondre à cette question.
Gemini Pro: Je ne trouve pas la réponse à votre question dans le texte fourni.
